In [6]:
from lxml import etree, html as lhtml
import requests
from tqdm.notebook import tqdm

## Этап 1. Получение ссылок на книги раздела

In [8]:
urls = []

for page_num in tqdm(range(1, 251)):
    r_post = requests.post(
        'https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php',
        data={
            'siteId': "s1",
            'template': ".default.5066805247fd90f4681c9458759019c47aeeee73c2035d99874d41cd9e7eb382",
            'parameters': "YToxNzM6e3M6MTg6IkNPTVBPTkVOVF9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxMToiSUJMT0NLX1RZUEUiO3M6NzoiY2F0YWxvZyI7czo5OiJJQkxPQ0tfSUQiO3M6MToiMiI7czoxMDoiU0VDVElPTl9JRCI7czo0OiI0NTc3IjtzOjEyOiJTRUNUSU9OX0NPREUiO047czoxMToiRklMVEVSX05BTUUiO3M6MTY6IkNBVEFMT0dfUFJPRFVDVFMiO3M6MTk6IklOQ0xVREVfU1VCU0VDVElPTlMiO3M6MToiQSI7czoxOToiU0hPV19BTExfV09fU0VDVElPTiI7czoxOiJOIjtzOjEzOiJDVVNUT01fRklMVEVSIjtzOjA6IiI7czoxODoiSElERV9OT1RfQVZBSUxBQkxFIjtzOjE6Ik4iO3M6MjU6IkhJREVfTk9UX0FWQUlMQUJMRV9PRkZFUlMiO3M6MToiWSI7czoxODoiRUxFTUVOVF9TT1JUX0ZJRUxEIjtzOjE1OiJQUk9QRVJUWV9SQVRJTkciO3M6MTg6IkVMRU1FTlRfU09SVF9PUkRFUiI7czo0OiJkZXNjIjtzOjE5OiJFTEVNRU5UX1NPUlRfRklFTEQyIjtzOjQ6Im5hbWUiO3M6MTk6IkVMRU1FTlRfU09SVF9PUkRFUjIiO3M6NDoiZGVzYyI7czoxNzoiT0ZGRVJTX1NPUlRfRklFTEQiO3M6MjoiaWQiO3M6MTc6Ik9GRkVSU19TT1JUX09SREVSIjtzOjM6ImFzYyI7czoxODoiT0ZGRVJTX1NPUlRfRklFTEQyIjtzOjU6InNob3dzIjtzOjE4OiJPRkZFUlNfU09SVF9PUkRFUjIiO3M6MzoiYXNjIjtzOjE4OiJQQUdFX0VMRU1FTlRfQ09VTlQiO3M6MjoiMjAiO3M6MTg6IkxJTkVfRUxFTUVOVF9DT1VOVCI7czoxOiI0IjtzOjIwOiJQUk9QRVJUWV9DT0RFX01PQklMRSI7YToyOntpOjA7czo4OiJBVFJfSVNCTiI7aToxO3M6MTM6IkFUUl9QVUJMSVNIRVIiO31zOjE3OiJPRkZFUlNfRklFTERfQ09ERSI7YTo2OntpOjA7czo0OiJOQU1FIjtpOjE7czoxMjoiUFJFVklFV19URVhUIjtpOjI7czoxNToiUFJFVklFV19QSUNUVVJFIjtpOjM7czoxMToiREVUQUlMX1RFWFQiO2k6NDtzOjE0OiJERVRBSUxfUElDVFVSRSI7aTo1O3M6MDoiIjt9czoyMDoiT0ZGRVJTX1BST1BFUlRZX0NPREUiO2E6Mjp7aTowO3M6NjoiTk9USUNFIjtpOjE7czowOiIiO31zOjEyOiJPRkZFUlNfTElNSVQiO3M6MToiMCI7czoyMDoiUFJPRFVDVF9ESVNQTEFZX01PREUiO3M6MToiWSI7czoxMzoiQUREX1BJQ1RfUFJPUCI7czoxNzoiQVRSX0VYVFJBX1BJQ1RVUkUiO3M6MTk6Ik9GRkVSX0FERF9QSUNUX1BST1AiO3M6MToiLSI7czoxNjoiT0ZGRVJfVFJFRV9QUk9QUyI7YToxOntpOjA7czo2OiJOT1RJQ0UiO31zOjIwOiJQUk9EVUNUX1NVQlNDUklQVElPTiI7czoxOiJZIjtzOjIxOiJTSE9XX0RJU0NPVU5UX1BFUkNFTlQiO3M6MToiWSI7czoxNDoiU0hPV19PTERfUFJJQ0UiO3M6MToiWSI7czoxNzoiU0hPV19NQVhfUVVBTlRJVFkiO3M6MToiQSI7czoyMToiQlhSX1NIT1dfTUFYX1FVQU5USVRZIjtzOjE6IkEiO3M6MTY6IlNIT1dfQ0xPU0VfUE9QVVAiO3M6MToiWSI7czoxMjoiTUVTU19CVE5fQlVZIjtzOjEyOiLQmtGD0L/QuNGC0YwiO3M6MjI6Ik1FU1NfQlROX0FERF9UT19CQVNLRVQiO3M6MTc6ItCSINC60L7RgNC30LjQvdGDIjtzOjE4OiJNRVNTX0JUTl9TVUJTQ1JJQkUiO3M6NDQ6ItCj0LLQtdC00L7QvNC40YLRjCDQviDQv9C+0YHRgtGD0L/Qu9C10L3QuNC4IjtzOjE1OiJNRVNTX0JUTl9ERVRBSUwiO3M6MzQ6ItCf0L7QtNGA0L7QsdC90LXQtSDQviDRgtC+0LLQsNGA0LUiO3M6MTg6Ik1FU1NfTk9UX0FWQUlMQUJMRSI7czoyNDoi0J3QtdGCINCyINC90LDQu9C40YfQuNC4IjtzOjExOiJTRUNUSU9OX1VSTCI7czozMToiL2NhdGFsb2ctcHJvZHVjdHMvI1NFQ1RJT05fSUQjLyI7czoxMDoiREVUQUlMX1VSTCI7czozMzoiL2NhdGFsb2ctcHJvZHVjdHMvI0VMRU1FTlRfQ09ERSMvIjtzOjE5OiJTRUNUSU9OX0lEX1ZBUklBQkxFIjtzOjEwOiJTRUNUSU9OX0lEIjtzOjEwOiJDQUNIRV9UWVBFIjtzOjE6IkEiO3M6MTA6IkNBQ0hFX1RJTUUiO3M6ODoiMzYwMDAwMDAiO3M6MTI6IkNBQ0hFX0ZJTFRFUiI7czoxOiJZIjtzOjEyOiJDQUNIRV9HUk9VUFMiO3M6MToiWSI7czo5OiJTRVRfVElUTEUiO3M6MToiWSI7czoxMzoiQlJPV1NFUl9USVRMRSI7czoxOiItIjtzOjEzOiJNRVRBX0tFWVdPUkRTIjtzOjE6Ii0iO3M6MTY6Ik1FVEFfREVTQ1JJUFRJT04iO3M6MToiLSI7czoxNzoiU0VUX0xBU1RfTU9ESUZJRUQiO3M6MToiWSI7czoyNDoiVVNFX01BSU5fRUxFTUVOVF9TRUNUSU9OIjtzOjE6IlkiO3M6MTg6IkFERF9TRUNUSU9OU19DSEFJTiI7czoxOiJOIjtzOjE1OiJBQ1RJT05fVkFSSUFCTEUiO3M6NjoiYWN0aW9uIjtzOjE5OiJQUk9EVUNUX0lEX1ZBUklBQkxFIjtzOjI6ImlkIjtzOjEwOiJQUklDRV9DT0RFIjthOjE6e2k6MDtzOjEwOiJCQVNFX1BSSUNFIjt9czoxNToiVVNFX1BSSUNFX0NPVU5UIjtzOjE6Ik4iO3M6MTY6IlNIT1dfUFJJQ0VfQ09VTlQiO3M6MDoiIjtzOjE3OiJQUklDRV9WQVRfSU5DTFVERSI7czoxOiJZIjtzOjE2OiJDT05WRVJUX0NVUlJFTkNZIjtzOjE6IlkiO3M6MTE6IkNVUlJFTkNZX0lEIjtzOjM6IlJVQiI7czoxMDoiQkFTS0VUX1VSTCI7czoyMDoiL3BlcnNvbmFsL2Jhc2tldC5waHAiO3M6MjA6IlVTRV9QUk9EVUNUX1FVQU5USVRZIjtzOjE6IlkiO3M6MjU6IlBST0RVQ1RfUVVBTlRJVFlfVkFSSUFCTEUiO3M6ODoicXVhbnRpdHkiO3M6MjQ6IkFERF9QUk9QRVJUSUVTX1RPX0JBU0tFVCI7czoxOiJZIjtzOjIyOiJQUk9EVUNUX1BST1BTX1ZBUklBQkxFIjtzOjQ6InByb3AiO3M6MjY6IlBBUlRJQUxfUFJPRFVDVF9QUk9QRVJUSUVTIjtzOjE6IlkiO3M6MTg6IlBST0RVQ1RfUFJPUEVSVElFUyI7YTo0OntpOjA7czoxMDoiQVRSX0FVVEhPUiI7aToxO3M6MTI6IkFUUl9MQU5HVUFHRSI7aToyO3M6NzoiQVRSX0FHRSI7aTozO3M6MTA6IkFUUl9PUklHSU4iO31zOjIyOiJPRkZFUlNfQ0FSVF9QUk9QRVJUSUVTIjthOjE6e2k6MDtzOjY6Ik5PVElDRSI7fXM6MjA6IkFERF9UT19CQVNLRVRfQUNUSU9OIjtOO3M6MTU6IkRJU1BMQVlfQ09NUEFSRSI7czoxOiJOIjtzOjE0OiJQQUdFUl9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxNzoiRElTUExBWV9UT1BfUEFHRVIiO3M6MToiWSI7czoyMDoiRElTUExBWV9CT1RUT01fUEFHRVIiO3M6MToiWSI7czoxMToiUEFHRVJfVElUTEUiO3M6MTI6ItCi0L7QstCw0YDRiyI7czoxNzoiUEFHRVJfU0hPV19BTFdBWVMiO3M6MToiTiI7czoyMDoiUEFHRVJfREVTQ19OVU1CRVJJTkciO3M6MToiTiI7czozMToiUEFHRVJfREVTQ19OVU1CRVJJTkdfQ0FDSEVfVElNRSI7czo1OiIzNjAwMCI7czoxNDoiUEFHRVJfU0hPV19BTEwiO3M6MToiTiI7czoyMjoiUEFHRVJfQkFTRV9MSU5LX0VOQUJMRSI7czoxOiJOIjtzOjE1OiJQQUdFUl9CQVNFX0xJTksiO047czoxNzoiUEFHRVJfUEFSQU1TX05BTUUiO047czo5OiJMQVpZX0xPQUQiO3M6MToiWSI7czoxODoiTUVTU19CVE5fTEFaWV9MT0FEIjtzOjIzOiLQn9C+0LrQsNC30LDRgtGMINC10YnRkSI7czoxNDoiTE9BRF9PTl9TQ1JPTEwiO3M6MToiTiI7czoxNDoiU0VUX1NUQVRVU180MDQiO3M6MToiWSI7czo4OiJTSE9XXzQwNCI7czoxOiJZIjtzOjg6IkZJTEVfNDA0IjtzOjg6Ii80MDQucGhwIjtzOjExOiJNRVNTQUdFXzQwNCI7czowOiIiO3M6MTU6IkNPTVBBVElCTEVfTU9ERSI7czoxOiJZIjtzOjI4OiJESVNBQkxFX0lOSVRfSlNfSU5fQ09NUE9ORU5UIjtzOjE6Ik4iO3M6MjI6IlVTRV9FTkhBTkNFRF9FQ09NTUVSQ0UiO3M6MToiWSI7czoxMjoiRU5MQVJHRV9QUk9QIjtzOjA6IiI7czoxMjoiQ09NUEFSRV9QQVRIIjtzOjUwOiIvY2F0YWxvZy1wcm9kdWN0cy9jb21wYXJlLnBocD9hY3Rpb249I0FDVElPTl9DT0RFIyI7czoxNjoiTUVTU19CVE5fQ09NUEFSRSI7czoxNjoi0KHRgNCw0LLQvdC40YLRjCI7czoxMjoiQ09NUEFSRV9OQU1FIjtzOjIwOiJDQVRBTE9HX0NPTVBBUkVfTElTVCI7czoyMjoiTUVTU19TSE9XX01BWF9RVUFOVElUWSI7czoxNDoi0J3QsNC70LjRh9C40LUiO3M6MjQ6IlJFTEFUSVZFX1FVQU5USVRZX0ZBQ1RPUiI7czoyOiIxMCI7czoyNzoiTUVTU19SRUxBVElWRV9RVUFOVElUWV9NQU5ZIjtzOjEwOiLQvNC90L7Qs9C+IjtzOjI2OiJNRVNTX1JFTEFUSVZFX1FVQU5USVRZX0ZFVyI7czo4OiLQvNCw0LvQviI7czoxNzoiUVVBTlRJVFlfSU5fU1RPQ0siO3M6MTc6ItCSINC90LDQu9C40YfQuNC4IjtzOjE4OiJRVUFOVElUWV9PVVRfU1RPQ0siO3M6MTc6ItCf0L7QtCDQt9Cw0LrQsNC3IjtzOjI0OiJISURFX1NFQ1RJT05fREVTQ1JJUFRJT04iO3M6MToiWSI7czoxMjoiVEhJU19VTklDX0lEIjtzOjk6IjFfc2VjdGlvbiI7czo2OiJSRUdJT04iO3M6MDoiIjtzOjIwOiJCWFJfQUpBWF9SRUdJT05fSU5GTyI7czowOiIiO3M6Mjg6IkJYUkVBRFlfRUxFTUVOVF9BRERDTEFTU19CSUciO3M6MDoiIjtzOjMwOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU01BTEwiO3M6MDoiIjtzOjMzOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU1RBTkRBUlQiO3M6MDoiIjtzOjI0OiJCWFJFQURZX0VMRU1FTlRfRFJBV19CSUciO3M6MTk6ImVjb21tZXJjZS5tMi5iaWcudjEiO3M6MjY6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NNQUxMIjtzOjIxOiJlY29tbWVyY2UubTIuc21hbGwudjEiO3M6Mjk6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NUQU5EQVJUIjtzOjE1OiJlY29tbWVyY2UubTIudjEiO3M6MzA6IkJYUkVBRFlfRUxFTUVOVF9FWFRfUEFSQU1TX0JJRyI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjMyOiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TTUFMTCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjM1OiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TVEFOREFSVCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjI4OiJCWFJFQURZX0xJU1RfTUFSS0VSX1RZUEVfQklHIjtzOjE1OiJyaWJib24udmVydGljYWwiO3M6MzA6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TTUFMTCI7czozOiJub3QiO3M6MzM6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TVEFOREFSVCI7czoxNToicmliYm9uLnZlcnRpY2FsIjtzOjMzOiJCWFJFQURZX0xJU1RfT1dOX01BUktFUl9VU0VfU01BTEwiO3M6MToiTiI7czozNjoiQlhSRUFEWV9MSVNUX09XTl9NQVJLRVJfVVNFX1NUQU5EQVJUIjtzOjE6Ik4iO3M6MjI6IkJYUkVBRFlfVVNFUl9UWVBFU19CSUciO3M6MToiTiI7czoyNDoiQlhSRUFEWV9VU0VSX1RZUEVTX1NNQUxMIjtzOjE6Ik4iO3M6Mjc6IkJYUkVBRFlfVVNFUl9UWVBFU19TVEFOREFSVCI7czoxOiJOIjtzOjI4OiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfQklHIjtzOjE6IlkiO3M6MzA6IkJYUkVBRFlfVVNFX0VMRU1FTlRDTEFTU19TTUFMTCI7czoxOiJZIjtzOjMzOiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfU1RBTkRBUlQiO3M6MToiWSI7czoyNjoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9CSUciO3M6MToiWSI7czoyODoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9TTUFMTCI7czoxOiJZIjtzOjMxOiJCWFJFQURZX1ZFUlRJQ0FMX0FMSUdOX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjI6IkJYUl9JTUdfTUFYX0hFSUdIVF9CSUciO3M6MzoiMTgwIjtzOjI0OiJCWFJfSU1HX01BWF9IRUlHSFRfU01BTEwiO3M6MjoiOTAiO3M6Mjc6IkJYUl9JTUdfTUFYX0hFSUdIVF9TVEFOREFSVCI7czozOiIxODAiO3M6MjE6IkJYUl9JTUdfTUFYX1dJRFRIX0JJRyI7czozOiIxODAiO3M6MjM6IkJYUl9JTUdfTUFYX1dJRFRIX1NNQUxMIjtzOjI6IjkwIjtzOjI2OiJCWFJfSU1HX01BWF9XSURUSF9TVEFOREFSVCI7czozOiIxODAiO3M6Mjg6IkJYUl9QUk9EVUNUX0JMT0NLU19PUkRFUl9CSUciO3M6NTE6InBpY3R1cmUscmF0aW5nLG5hbWUsYXJ0aWNsZSxwcmV2aWV3dGV4dCxhY3Rpb250aW1lciI7czozMzoiQlhSX1BST0RVQ1RfQkxPQ0tTX09SREVSX1NUQU5EQVJUIjtzOjM5OiJwaWN0dXJlLG5hbWUsYWN0aW9udGltZXIsYXJ0aWNsZSxyYXRpbmciO3M6MjU6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9CSUciO3M6MToiTiI7czoyNzoiQlhSX1NIT1dfQUNUSU9OX1RJTUVSX1NNQUxMIjtzOjE6Ik4iO3M6MzA6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9TVEFOREFSVCI7czoxOiJOIjtzOjIwOiJCWFJfU0hPV19BUlRJQ0xFX0JJRyI7czoxOiJZIjtzOjIyOiJCWFJfU0hPV19BUlRJQ0xFX1NNQUxMIjtzOjE6IlkiO3M6MjU6IkJYUl9TSE9XX0FSVElDTEVfU1RBTkRBUlQiO3M6MToiWSI7czoyNToiQlhSX1NIT1dfUFJFVklFV19URVhUX0JJRyI7czoxOiJZIjtzOjE5OiJCWFJfU0hPV19SQVRJTkdfQklHIjtzOjM6ImF2ZyI7czoyMToiQlhSX1NIT1dfUkFUSU5HX1NNQUxMIjtzOjM6ImF2ZyI7czoyNDoiQlhSX1NIT1dfUkFUSU5HX1NUQU5EQVJUIjtzOjM6ImF2ZyI7czoxOToiQlhSX1NIT1dfU0xJREVSX0JJRyI7czoxOiJOIjtzOjIxOiJCWFJfU0hPV19TTElERVJfU01BTEwiO3M6MToiWSI7czoyNDoiQlhSX1NIT1dfU0xJREVSX1NUQU5EQVJUIjtzOjE6Ik4iO3M6Mjc6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX0JJRyI7czo2OiJzcXVhcmUiO3M6Mjk6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX1NNQUxMIjtzOjY6InNxdWFyZSI7czozMjoiQlhSX1NLVV9QUk9QU19TSE9XX1RZUEVfU1RBTkRBUlQiO3M6Njoic3F1YXJlIjtzOjIzOiJCWFJfU0xJREVSX0lOVEVSVkFMX0JJRyI7czo0OiIzMDAwIjtzOjI4OiJCWFJfU0xJREVSX0lOVEVSVkFMX1NUQU5EQVJUIjtzOjQ6IjMwMDAiO3M6Mjg6IkJYUl9TTElERVJfUFJPR1JFU1NfU1RBTkRBUlQiO3M6MToiTiI7czoyODoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX0JJRyI7czoxOiJZIjtzOjMwOiJCWFJfVElMRV9TSE9XX1BST1BFUlRJRVNfU01BTEwiO3M6MToiTiI7czozMzoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjE6IkJYUl9VU0VfRkFTVF9WSUVXX0JJRyI7czoxOiJOIjtzOjIzOiJCWFJfVVNFX0ZBU1RfVklFV19TTUFMTCI7czoxOiJOIjtzOjI2OiJCWFJfVVNFX0ZBU1RfVklFV19TVEFOREFSVCI7czoxOiJOIjtzOjIyOiJNRVNTX0JUTl9GQVNUX1ZJRVdfQklHIjtzOjMxOiLQkdGL0YHRgtGA0YvQuSDQv9GA0L7RgdC80L7RgtGAIjtzOjI0OiJNRVNTX0JUTl9GQVNUX1ZJRVdfU01BTEwiO3M6MzE6ItCR0YvRgdGC0YDRi9C5INC/0YDQvtGB0LzQvtGC0YAiO3M6Mjc6Ik1FU1NfQlROX0ZBU1RfVklFV19TVEFOREFSVCI7czozMToi0JHRi9GB0YLRgNGL0Lkg0L/RgNC+0YHQvNC+0YLRgCI7czoyMDoiUFJPRFVDVF9ST1dfVkFSSUFOVFMiO3M6MTY2OiJbeydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX1dIjtzOjEzOiJQUk9QRVJUWV9DT0RFIjthOjI6e2k6MDtzOjg6IkFUUl9JU0JOIjtpOjE7czoxMzoiQVRSX1BVQkxJU0hFUiI7fXM6MTc6IkNVUlJFTlRfQkFTRV9QQUdFIjtzOjU1OiIvY2F0YWxvZy1wcm9kdWN0cy80NTc3Lz9zb3J0PVBST1BFUlRZX1JBVElORyZvcmRlcj1kZXNjIjtzOjExOiJQQVJFTlRfTkFNRSI7czoyMzoiYnhyZWFkeS5tYXJrZXQyOmNhdGFsb2ciO3M6MjA6IlBBUkVOVF9URU1QTEFURV9OQU1FIjtzOjEzOiJyZWxvZC1jYXRhbG9nIjtzOjIwOiJQQVJFTlRfVEVNUExBVEVfUEFHRSI7czo3OiJzZWN0aW9uIjtzOjEzOiJHTE9CQUxfRklMVEVSIjthOjA6e319.b03951e04cbd1e9052c9d580f4da66bc7c86d018e24aafd38b4155c6047efdd8",
            'action': "showMore",
            'PAGEN_1': str(page_num)
        })
    page = r_post.text
    tree = lhtml.fromstring(page)
    urls_from_page = \
        tree.xpath('//a[contains(@class, "bxr-item-image-wrap")]/attribute::href', \
                   smart_strings=False)
    urls_from_page = \
        list(map(lambda s: 'https://shop.relod.ru' + s, urls_from_page))
    urls += urls_from_page
    
assert len(urls) == len(set(urls))

In [9]:
with open('data/urls.txt', 'w') as fout:
    for url in tqdm(urls):
        print(url, file=fout)

## Этап 2. Получение информации о книгах

In [10]:
import re
import gzip
import json
import codecs
import sys
from multiprocessing.dummy import Pool, Queue

In [11]:
def get_page(url, n_attempts=5, t_sleep=1):
    for _ in range(n_attempts):
        r_get = requests.get(url)
        if r_get.ok:
            break
    if not r_get.ok:
        print(f'Cannot get page {url}', file=sys.stderr)
        return 
    page = r_get.text
    return page


def process_page(url):    
    page = get_page(url)
    if page is None:
        return
    tree = lhtml.fromstring(page)
    
    book_info = {}
    
    book_info['url'] = url
    
    title = tree.xpath('//h1[@itemprop="name"]/text()')[0]
    book_info['Название'] = title
    
    rating_val = tree.xpath('//meta[@itemprop="ratingValue"]//attribute::content')
    rating_val = float(rating_val[0]) if rating_val else 0
    book_info['Оценка'] = rating_val
    
    rating_cnt = tree.xpath('//meta[@itemprop="ratingCount"]//attribute::content')
    rating_cnt = int(rating_cnt[0]) if rating_cnt else 0
    book_info['Число голосов'] = rating_cnt
    
    availability = tree.xpath('//div[@itemprop="availability"]//text()')[0]
    book_info['Наличие'] = availability
    
    price = tree.xpath('//meta[@itemprop="price"]/attribute::content')[0]
    price = float(price)
    book_info['Цена'] = price
    
    outer = re.findall(r'"PRICE"\s?:\s?(\d+(.\d+)?)', page)
    if outer:
        discount_price = float(outer[0][0])
        book_info['Цена (скидка)'] = discount_price
    
    labels = tree.xpath('//div[@class="bxr-ribbon-marker-vertical"]/span/i/text()') \
        + tree.xpath('//div[@class="bxr-ribbon-marker-vertical"]/a/text()')
    book_info['Метки'] = labels
    
    description = tree.xpath('//div[@class="bxr-detail"]//text()')
    description = ' '.join(description).strip()
    description = re.sub('\s+', ' ', description)
    description = description.replace('\x92', '\'').replace('\x85', '...')
    book_info['Описание'] = description
    
    imgs = tree.xpath('//img[@class="bxr-zoom-img lazy"]/attribute::data-src')
    imgs = list(map(lambda s: 'https:' + s, imgs))
    book_info['Иллюстрации'] = imgs
    
    trs = tree.xpath('//table[@class="bxr-props-table"]//tr')
    table = {}
    for tr in trs:
        name = tr.xpath('./td[@class="bxr-props-name"]/span/text()')[0]
        value = tr.xpath('./td[@class="bxr-props-data"]/*[1]//text()')[0]
        table[name] = value
    book_info.update(table)
    
    return book_info

In [12]:
with open('data/urls.txt') as fin:
    urls = [url[:-2] for url in fin]

In [13]:
queue = Queue()
for url in urls:
    queue.put(url)
    
def process_page_wrapper(i):        
    with gzip.open(f'data/cards/part_{i:05d}.jsonl.gz', mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            url = queue.get()
            try:
                record = process_page(url)
            except Exception:
                with lock:
                    print(f'Got exception while processing page {url}', file=sys.stderr)
            # if record is None, this means we couldn't get the page, 
            # and this trouble is already logged
            if record:
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
     
            with lock:
                pbar.update(1)

with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

## Этап 3. Объединение результатов

In [14]:
import pandas as pd
import os
from itertools import chain
from contextlib import ExitStack
from typing import Generator, Dict, Any

In [15]:
def records_reader(dirname: str='data/cards') -> Generator[Dict[str, Any], None, None]:
    with ExitStack() as stack:
        f_jsons = (
            stack.enter_context(gzip.open(dirname + '/' + fname, mode='rb')) 
            for fname in os.listdir(dirname)
            if fname.endswith('.jsonl.gz')
        )
        f_jsons = map(codecs.getreader('utf8'), f_jsons)

        records = (
            (record_str for record_str in f_json)
            for f_json in f_jsons
        )
        records = (
            map(json.loads, strs)
            for strs in records
        )
        yield from chain.from_iterable(records)

In [16]:
df = pd.DataFrame(records_reader('data/cards'))
df.to_csv('hw_3.csv', index=False)

In [17]:
pd.set_option('display.max_columns', 30)

In [18]:
df.head()

,ISBN,url,Автор,Вес (грамм),Возрастные ограничения,Год издания,Издание,Издатель,Иллюстрации,Метки,Название,Наличие,Носитель,Обзор/Ролик,Описание,Оценка,Размер (мм),Рейтинг,Рекомендованный возраст,Серия,Страна происхождения,Страницы,Тип продукта,Формат,Цена,Цена (скидка),Число голосов,Читательская аудитория,Язык
0,9781862073005,https://shop.relod.ru/catalog-products/the_ess...,Chekhov Anton,278,16+,1999,NaN,Granta,[https://opt-1458870.ssl.1c-bitrix-cdn.ru/uplo...,[],The Essential Tales of Chekhov,В наличии,NaN,NaN,"In this collection of 20 short stories, the ed...",5.0,198(д) х 129(ш) х 26(в),3.3,16+,NaN,Соединенное Королевство,368,NaN,Paperback,969.0,NaN,1,NaN,English
1,9781421598246,https://shop.relod.ru/catalog-products/tokyo-g...,NaN,268,16+,2019,NaN,Viz Media,[https://opt-1458870.ssl.1c-bitrix-cdn.ru/uplo...,[],"Tokyo Ghoul: re, Vol. 9",Под заказ,NaN,NaN,"In the world of Tokyo Ghoul, sometimes the onl...",4.0,210(д) х 146(ш) х 18(в),3.2,16+,NaN,NaN,248,NaN,Paperback,1267.0,NaN,1,NaN,English
2,9780751550658,https://shop.relod.ru/catalog-products/zoya,Steel Danielle,220,16+,2012,NaN,Sphere,[https://opt-1458870.ssl.1c-bitrix-cdn.ru/uplo...,[],Zoya,Под заказ,NaN,NaN,One woman's odyssey through a century of turmo...,0.0,197(д) х 126(ш) х 24(в),NaN,16+,NaN,NaN,416,NaN,Paperback,1113.0,NaN,0,NaN,English
3,9781782060833,https://shop.relod.ru/catalog-products/zero_si...,Lewis Damien,323,16+,2014,NaN,Quercus,[https://opt-1458870.ssl.1c-bitrix-cdn.ru/uplo...,[],Zero Six Bravo,Под заказ,NaN,NaN,Sunday Times No.1 bestseller. They were brande...,0.0,198(д) х 132(ш) х 29(в),NaN,16+,NaN,NaN,336,NaN,Paperback,1236.0,NaN,0,NaN,English
4,9781444761436,https://shop.relod.ru/catalog-products/z_a_nov...,Fowler Therese Anne,259,16+,2013,NaN,Two Roads,[https://opt-1458870.ssl.1c-bitrix-cdn.ru/uplo...,[],Z: A Novel of Zelda Fitzgerald,Под заказ,NaN,NaN,"'If ever a couple ...became an era, it was F S...",0.0,197(д) х 132(ш) х 24(в),NaN,16+,NaN,NaN,384,NaN,Paperback,1236.0,NaN,0,NaN,English
